In [46]:
import numpy as np

In [47]:
names = ["review_id","rating","review_title",'review_text','username','user_location','hotel_id','date_stayed','review_date']

In [48]:
import pandas as pd
df = pd.read_csv('data2.zip',sep=";;",error_bad_lines=False,names= names)
df.sample(15)
df = df.dropna(subset=['rating'])

/home/mg/.anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
Skipping line 50647: Expected 9 fields in line 50647, saw 11. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 71068: Expected 9 fields in line 71068, saw 10. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 73612: Expected 9 fields in line 73612, saw 11. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 104136: Expected 9 fields in line 104136, saw 10. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 122095: Expected 9 fields in line 122095, saw 13. Error could pos

## Encoding user

In [49]:
df['username'] = df['username'].fillna("Anon")

In [50]:
from sklearn import preprocessing

le_user  = preprocessing.LabelEncoder()
le_user.fit(df['username'].values)



LabelEncoder()

In [51]:
df['username'] = le_user.transform(df['username'].values)

In [52]:
USER_COUNT = len(le_user.classes_)

# Encoding hotel

In [53]:
df['hotel_id'] = df['hotel_id'].fillna('UNK') 

In [54]:
le_hotel =  preprocessing.LabelEncoder()
le_hotel.fit(df['hotel_id'].values)


LabelEncoder()

In [55]:
df['hotel_id'] = le_hotel.transform(df['hotel_id'].values)

In [56]:
HOTEL_COUNT = len(le_hotel.classes_)


## Score harmonization

In [57]:
df['rating'] = df['rating'].fillna(-1) 

## Matrix factorization

In [58]:
from surprise import SVD,NMF,SVDpp
from surprise import Dataset
from surprise import Reader
from surprise import NormalPredictor



from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy




## Test

In [59]:



rating_df = pd.DataFrame()
rating_df["rating"] = df['rating']
rating_df["username"] = df["username"]
rating_df["hotel_id"] = df["hotel_id"]

In [60]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['username', 'hotel_id', 'rating']], reader)


In [61]:
# sample random trainset and testset
# test set is made of 15% of the ratings.
trainset, testset = train_test_split(data, test_size=.15)

algos = [(SVD(),"SVD"),(NMF(),"NMF")]

for algo,name in algos:
    algo.fit(trainset)
    predictions = algo.test(testset)
    
    print(name,accuracy.rmse(predictions))


RMSE: 1.0581
SVD 1.0580697005584871
RMSE: 1.2293
NMF 1.2292878303233175


KeyboardInterrupt: 

## Test

In [ ]:

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n




In [ ]:
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
